In [56]:
import matplotlib
matplotlib.use('pdf')

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
plt.style.use('default')

In [58]:
def human_format(x):
    sigpart = 0
    suffix = ""
    if x < 1000:
        return "{:.3g}".format(x)
    elif x < 1000000:
        sigpart = x / 1000
        suffix = "k"
    elif x < 1000000000:
        sigpart = x / 1000000
        suffix = "M"
    else:
        sigpart = x / 1000000000
        suffix = "B"
    return "{:.3g}{}".format(sigpart, suffix)
def get_algorithm(path):
    algs = ["rkde", "ic2", "sklearn", "nocut", "simple"]
    for alg in algs:
        if alg in path:
            return alg
def get_dataset(path):
    datasets = {
        "bgauss": "gauss",
        "hep": "hep",
        "tmy3_4": "tmy3_4",
        "tmy3.": "tmy3",
        "mnist_64": "mnist",
        "home": "home",
    }
    for dataset in datasets.keys():
        if dataset in path:
            return datasets[dataset]
markers = {
    "ic2": "o",
    "simple": "^",
    "sklearn": "s",
    "rkde": "d",
    "nocut": "8",
    "ks": "*",
}

In [59]:
df = pd.read_csv("../experiments/d_raiders/scale_d.csv")

In [60]:
df["r_algorithm"] = df["out_path"].map(get_algorithm)
df["r_dataset"] = df["out_path"].map(get_dataset)
df["throughput"] = (
    1000 * df["num_test"] / df["test_time"]
)
df["a_throughput"] = (
    1000 * df["num_train"] 
    / (df["test_time"]*df["num_train"]/df["num_test"] + df["train_time"])
)

In [61]:
df = df.sort_values(by=["r_algorithm", "dim"])

In [62]:
algorder = ["ic2", "simple", "sklearn", "rkde"]

In [63]:
matplotlib.rcParams.update({'font.size': 11})
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
fig = plt.figure(figsize=(6,3.5),dpi=300)
ax = plt.subplot(111)
for i,alg in enumerate(algorder):
    cur_df = df[df["r_algorithm"] == alg]
    ax.loglog(
        cur_df["dim"],
        cur_df["throughput"],
        marker=markers[alg],
        linewidth=1.0,
        markersize=4.0,
        label=alg,
        basex=2,
    )
majorFormatter = FormatStrFormatter('%d')
ax.xaxis.set_major_formatter(majorFormatter)
ax.set_xlabel("Dataset Dimension")
ax.set_ylabel("Query Throughput")
ax.set_title("Adjusting Dimension, mnist, n={}".format(
        human_format(df.num_train.iloc[0]),
        df.dim.iloc[0]))
ax.legend(loc=0, ncol=2)
fig.tight_layout()
fig.savefig("d_scale.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
